# On-Chain Arbitrage Routing

- Pre-processing arbitrage routes for a single chain
- Assets can be identified by address on-chain, and all dex pairs can be directly interacted with without bridging/depositing/withdrawing
- Assumed all long holdings are USDT, therefore all arbitrages must begin and terminate with USDT

In [54]:
import pandas as pd
import numpy as np
import json

usdt_address = r"0x382bB369d343125BfB2117af9c149795C6C65C50".lower()
chain_name = 'okx'
product_ids = ['aiSwap', 'cherrySwap', 'jswapFinance', 'kSwap', 'pandaSwap']

In [55]:
def load_pairs_for_product(chain_name, product_id):
  filename = f'input/pairs/{chain_name}-{product_id}.json'
  with open(filename, 'r') as f:
    data_json = json.load(f)
  df = pd.json_normalize(data_json['data'])
  df = df.drop('level_0', axis = 1)
  df = df.groupby(['index']).max().reset_index()
  df['productId'] = product_id
  return df

def load_pairs_for_products(chain_name, product_ids):
  return pd.concat(map(lambda product_id: load_pairs_for_product(chain_name, product_id), product_ids))

df_pairs = load_pairs_for_products(chain_name, product_ids)
df_pairs.head()

,index,address,token_0,name_0,symbol_0,decimals_0,token_1,name_1,symbol_1,decimals_1,productId
0,1,0xf6a14934ed35e1e2bb448555baaf37adb91297ed,0x3212606f74cc59656e1ec6f587fca61ba3b85eb0,SFGK,SFGK,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
1,2,0x7b44eef5060a4b1007705cfccb40a866100180ff,0x21cde7e32a6caf4742d00d44b07279e7596d26b9,DAIK,DAIK,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
2,3,0x44e8a805f8c06de4dcedf3cd169688a6a5fceaba,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
3,6,0x3cca939888e796c104434bd120a3ba0d10f1834b,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,aiSwap
4,14,0x0c6cdba862b6c5a82f7297934043a059e0ef2c54,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,aiSwap


### Number of pairs per product

In [56]:
df_pairs.groupby(by = 'productId').count()['index']

productId
aiSwap            6
cherrySwap      114
jswapFinance    159
kSwap            32
pandaSwap        14
Name: index, dtype: int64

## USDT Pair Routes

Simply selects all the pairs with a USDT side, and groups them by the other side. 

### 1. Get all pairs against USDT

In [57]:
df_pairs_usdt = df_pairs[(df_pairs['token_0'] == usdt_address) | (df_pairs['token_1'] == usdt_address)]
df_pairs_usdt.head()

,index,address,token_0,name_0,symbol_0,decimals_0,token_1,name_1,symbol_1,decimals_1,productId
2,3,0x44e8a805f8c06de4dcedf3cd169688a6a5fceaba,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
3,6,0x3cca939888e796c104434bd120a3ba0d10f1834b,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,aiSwap
4,14,0x0c6cdba862b6c5a82f7297934043a059e0ef2c54,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,aiSwap
0,0,0xf3098211d012ff5380a03d80f150ac6e5753caa8,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,cherrySwap
1,1,0x94e01843825ef85ee183a711fa7ae0c5701a731a,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,cherrySwap


### 2. Get all USDT pairs with at least one other pair between the same assets

Note:

- The other pairs can have the non-USDT token on either side (e.g. be USDT/X or X/USDT)
- They may be on the same dex, or a different dex

In [58]:
df_pairs_usdt_with_other = df_pairs_usdt.copy()
df_pairs_usdt_with_other['token_other'] = df_pairs_usdt_with_other.apply(lambda row: row['token_1'] if row['token_0'] == usdt_address else row['token_0'], axis = 1)
df_pairs_usdt_with_other['symbol_other'] = df_pairs_usdt_with_other.apply(lambda row: row['symbol_1'] if row['token_0'] == usdt_address else row['symbol_0'], axis = 1)

df_pairs_usdt_by_token = df_pairs_usdt_with_other.groupby('token_other')['token_other'].size()

set_pairs_usdt_addresses = set(df_pairs_usdt_by_token[df_pairs_usdt_by_token > 1].index.values)

df_pairs_usdt_arbs = df_pairs_usdt_with_other[df_pairs_usdt_with_other.apply(lambda row: row['token_other'] in set_pairs_usdt_addresses, axis = 1)]
df_pairs_usdt_arbs = df_pairs_usdt_arbs.sort_values('token_other')

del df_pairs_usdt_with_other
del df_pairs_usdt_by_token
del set_pairs_usdt_addresses

df_pairs_usdt_arbs.head()

,index,address,token_0,name_0,symbol_0,decimals_0,token_1,name_1,symbol_1,decimals_1,productId,token_other,symbol_other
25,149,0xb6d297c6b9a7863ed67b67211ca3e356a5f02b92,0x00505505a7576d6734704fabb16f41924e3e384b,TRXK,TRXK,18.0,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,jswapFinance,0x00505505a7576d6734704fabb16f41924e3e384b,TRXK
30,557,0x9b287c7b4b4ae3843a129815b7ad417e19de8bff,0x00505505a7576d6734704fabb16f41924e3e384b,TRXK,TRXK,18.0,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,cherrySwap,0x00505505a7576d6734704fabb16f41924e3e384b,TRXK
26,150,0x50b942a347564063b40e42e72a86650d86cce776,0x18d103b7066aeedb6005b78a462ef9027329b1ea,BCHK,BCHK,18.0,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,jswapFinance,0x18d103b7066aeedb6005b78a462ef9027329b1ea,BCHK
32,559,0x453377749a1d515eb6d99f1bdfd987785e55943e,0x18d103b7066aeedb6005b78a462ef9027329b1ea,BCHK,BCHK,18.0,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,cherrySwap,0x18d103b7066aeedb6005b78a462ef9027329b1ea,BCHK
14,55,0xe9313b7dea9cbabd2df710c25bef44a748ab38a9,0x21cde7e32a6caf4742d00d44b07279e7596d26b9,DAIK,DAIK,18.0,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,jswapFinance,0x21cde7e32a6caf4742d00d44b07279e7596d26b9,DAIK


### Analysis: How many distinct pairs are there?

In [74]:
df_pairs_usdt_arbs_grouped = df_pairs_usdt_arbs.groupby(by = ['token_other', 'symbol_other'])

df_pairs_usdt_arbs_groups = pd.DataFrame(df_pairs_usdt_arbs_grouped.size(), columns = ['count'])
df_pairs_usdt_arbs_groups['productIds'] = df_pairs_usdt_arbs_grouped['productId'].apply(lambda product_ids: ', '.join(sorted(product_ids)))
df_pairs_usdt_arbs_groups = df_pairs_usdt_arbs_groups.sort_values(by = 'count', ascending = False)

del df_pairs_usdt_arbs_grouped

df_pairs_usdt_arbs_groups

,,count,productIds
token_other,symbol_other,,
0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,WOKT,5,"aiSwap, cherrySwap, jswapFinance, kSwap, panda..."
0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,5,"aiSwap, cherrySwap, jswapFinance, kSwap, panda..."
0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,5,"aiSwap, cherrySwap, jswapFinance, kSwap, panda..."
0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,4,"cherrySwap, jswapFinance, kSwap, pandaSwap"
0x59d226bb0a4d74274d4354ebb6a0e1a1aa5175b6,UNIK,3,"cherrySwap, jswapFinance, kSwap"
0xa715f624eb32c6c960b0e044b10cde585d5f5984,XRPK,3,"cherrySwap, jswapFinance, kSwap"
0xfa520efc34c81bfc1e3dd48b7fe9ff326049f986,LTCK,3,"cherrySwap, jswapFinance, kSwap"
0x5ab622494ab7c5e81911558c9552dbd517f403fb,Celt,2,"cherrySwap, jswapFinance"
0x79b627bc95fa5b36eca53eb39c3cdf43aafdd10f,ORG,2,"cherrySwap, kSwap"


## USDT Triangular Arbitrage Routes

https://en.wikipedia.org/wiki/Triangular_arbitrage

### 1. Get all pairs against USDT

In [60]:
df_pairs_usdt = df_pairs[(df_pairs['token_0'] == usdt_address) | (df_pairs['token_1'] == usdt_address)]
df_pairs_usdt.head()

,index,address,token_0,name_0,symbol_0,decimals_0,token_1,name_1,symbol_1,decimals_1,productId
2,3,0x44e8a805f8c06de4dcedf3cd169688a6a5fceaba,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
3,6,0x3cca939888e796c104434bd120a3ba0d10f1834b,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,aiSwap
4,14,0x0c6cdba862b6c5a82f7297934043a059e0ef2c54,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,aiSwap
0,0,0xf3098211d012ff5380a03d80f150ac6e5753caa8,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,cherrySwap
1,1,0x94e01843825ef85ee183a711fa7ae0c5701a731a,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,cherrySwap


### 2. Get the set of all tokens with a USDT pair

In [61]:
set_tokens_with_usdt_pairs = set(pd.concat([df_pairs_usdt['token_0'], df_pairs_usdt['token_1']], axis = 0))
set_tokens_with_usdt_pairs.discard(usdt_address)

print(f'Number of tokens = {len(set_tokens_with_usdt_pairs)}')

Number of tokens = 106


### 3. Get all pairs NOT against USDT

In [62]:
df_pairs_non_usdt = df_pairs[(df_pairs['token_0'] != usdt_address) & (df_pairs['token_1'] != usdt_address)]
df_pairs_non_usdt.head()

,index,address,token_0,name_0,symbol_0,decimals_0,token_1,name_1,symbol_1,decimals_1,productId
0,1,0xf6a14934ed35e1e2bb448555baaf37adb91297ed,0x3212606f74cc59656e1ec6f587fca61ba3b85eb0,SFGK,SFGK,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
1,2,0x7b44eef5060a4b1007705cfccb40a866100180ff,0x21cde7e32a6caf4742d00d44b07279e7596d26b9,DAIK,DAIK,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,aiSwap
5,15,0x13b3e85f115a2938be5345c62e68bba5c858f71d,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,aiSwap
3,3,0x8e68c0216562bcea5523b27ec6b9b6e1cccbbf88,0x8179d97eb6488860d816e3ecafe694a4153f216c,CherrySwap Token,CHE,18.0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,cherrySwap
7,26,0x351e76d3339a143eb7b1709b4514404022ccc35e,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,0xda3a3002c7af6b465b3bf06ce5eac974f2515b5f,hard to write contract,MiNiCHE,18.0,cherrySwap


### 4. Forall non-USDT pairs, find those in which both sides have a USDT pair

In [143]:
token_symbol_by_address = pd.concat([
  df_pairs_non_usdt[['token_0', 'symbol_0']].rename({'token_0':'token','symbol_0':'symbol'}, axis = 1).set_index('token'),
  df_pairs_non_usdt[['token_1', 'symbol_1']].rename({'token_1':'token','symbol_1':'symbol'}, axis = 1).set_index('token')
]).drop_duplicates().to_dict()['symbol']

df_pairs_tri_arbs = df_pairs_non_usdt[(df_pairs_non_usdt['token_0'].isin(set_tokens_with_usdt_pairs)) & (df_pairs_non_usdt['token_1'].isin(set_tokens_with_usdt_pairs))].copy()
df_pairs_tri_arbs['group_key'] = df_pairs_tri_arbs.apply(lambda row: ','.join(sorted([row['token_0'], row['token_1']])), axis = 1)

df_pairs_tri_arbs_grouped = df_pairs_tri_arbs.groupby(by = ['group_key'])

df_pairs_tri_arbs_groups = pd.DataFrame(df_pairs_tri_arbs_grouped.size(), columns = ['count'])
df_pairs_tri_arbs_groups['pairAddresses'] = df_pairs_tri_arbs_grouped['address'].apply(lambda pair_addresses: '[' + ', '.join(sorted(pair_addresses)) + ']')
df_pairs_tri_arbs_groups['productIds'] = df_pairs_tri_arbs_grouped['productId'].apply(lambda product_ids: '[' + ', '.join(sorted(product_ids)) + ']')
df_pairs_tri_arbs_groups['token_0'] = df_pairs_tri_arbs_grouped.apply(lambda g: g.name.split(',')[0])
df_pairs_tri_arbs_groups['symbol_0'] = df_pairs_tri_arbs_groups.apply(lambda row: token_symbol_by_address[row['token_0']], axis = 1)
df_pairs_tri_arbs_groups['token_1'] = df_pairs_tri_arbs_grouped.apply(lambda g: g.name.split(',')[1])
df_pairs_tri_arbs_groups['symbol_1'] = df_pairs_tri_arbs_groups.apply(lambda row: token_symbol_by_address[row['token_1']], axis = 1)
df_pairs_tri_arbs_groups['arbLabel'] = df_pairs_tri_arbs_groups.apply(lambda row: f'USDT=>{row["symbol_0"]}=>{row["symbol_1"]}=>USDT', axis = 1)


df_pairs_tri_arbs_groups = df_pairs_tri_arbs_groups.sort_values(by = 'count', ascending = False)
df_pairs_tri_arbs_groups = df_pairs_tri_arbs_groups.reset_index().drop(columns = ['group_key'])

del token_symbol_by_address
del df_pairs_tri_arbs_grouped

df_pairs_tri_arbs_groups

,count,pairAddresses,productIds,token_0,symbol_0,token_1,symbol_1,arbLabel
0,3,"[0xaea843a6715d091f7a5644ed4fcca479820bd18a, 0...","[cherrySwap, jswapFinance, kSwap]",0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,WOKT,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,USDT=>WOKT=>OKB=>USDT
1,3,"[0x24aa7ec16b1965bc3e1c68de80298b16cbc6ef92, 0...","[cherrySwap, jswapFinance, kSwap]",0x39071d9e7a52e8685f1385d6fa17babfc58ce9c3,KLT,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,WOKT,USDT=>KLT=>WOKT=>USDT
2,3,"[0x1290e0cd4ed25919bc7d5cf27c9a62199b1de10e, 0...","[cherrySwap, jswapFinance, kSwap]",0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,USDT=>BTCK=>ETHK=>USDT
3,2,"[0x579c3b456efe2591173cc3accbaba530e318e89f, 0...","[cherrySwap, jswapFinance]",0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,WOKT,0xdcac52e001f5bd413aa6ea83956438f29098166b,USDK,USDT=>WOKT=>USDK=>USDT
4,1,[0xc577dd5fb0fcbcd65edf922cf257ec0cfe42f88a],[kSwap],0x0b7035b1b32b4a8be30055efc3909e4358000527,CASH,0x43a7d5573d0c3d0fac233c0cd1c9a73c8ce9ff72,SHARE,USDT=>CASH=>SHARE=>USDT
5,1,[0x4225f964eeac8c022a39e5fd1c23791e113b0c75],[jswapFinance],0x5fac926bf1e638944bb16fb5b787b5ba4bc85b0a,JF,0x8da6da06a4902015cfe823b24c5ed486016ba3cb,YYDS,USDT=>JF=>YYDS=>USDT
6,1,[0x562f8cf1e7a5f72e01452c6bceebd4cd3399cb94],[cherrySwap],0xdcac52e001f5bd413aa6ea83956438f29098166b,USDK,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,USDT=>USDK=>OKB=>USDT
7,1,[0x9d2220f710bdf68ac2bc6d5d27f0eda629d0a638],[jswapFinance],0xc946daf81b08146b1c7a8da2a851ddf2b3eaaf85,USDC,0xdcac52e001f5bd413aa6ea83956438f29098166b,USDK,USDT=>USDC=>USDK=>USDT
8,1,[0x13b3e85f115a2938be5345c62e68bba5c858f71d],[aiSwap],0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,WOKT,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,USDT=>WOKT=>ETHK=>USDT
9,1,[0xa25da5a44a65ee9bd4ea61f946cbcf15512fd52e],[kSwap],0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,WOKT,0xab0d1578216a545532882e420a8c61ea07b00b12,KST,USDT=>WOKT=>KST=>USDT
